# TfL Assistant using OpenAI API Function calling and TfL API
This notebook demonstrates how to use OpenAI API function calling to call TfL API and get the status of a London Underground line.

## Prerequisites
1. [OpenAI API key](https://beta.openai.com/)
2. [TfL API key](https://api-portal.tfl.gov.uk/login)
3. [PowerShell](https://docs.microsoft.com/en-us/powershell/scripting/install/installing-powershell?view=powershell-7.1)

## Sample flow
1. Get query from user
2. Call OpenAI API with the query and also tell OpenAI that we have a function in our code to call TfL API
3. OpenAI will see the user query and if relevant, will tell us to call the TfL API function in our code
4. Call TfL API function in our code
5. Return the TfL API response and user's initial message to OpenAI
6. OpenAI will analyse the TfL API response along with user's initial message and return a response

## Global variables for OpenAI API

Replace the following variables with your own values:

In [ ]:
$openai_api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx";
$openai_api_endpoint = "https://api.openai.com/v1/chat/completions";
$openai_api_model = "gpt-3.5-turbo-0613";

## Functions
Lets look at the functions that we will be using in this notebook.

### Get-TubeStatus
This function calls TfL API and gets the status of a London Underground line.

In [ ]:
function Get-TubeStatus {
    param(
        [Parameter(Mandatory = $true)]
        [string]$line
    )

    $tfl_api_endpoint = "https://api.tfl.gov.uk/Line/$line/Status";
    $response = Invoke-WebRequest -Uri $tfl_api_endpoint -Method Get
    return $response;

}

### Invoke-OpenAI
This function calls OpenAI API and gets the response from the API.

In [ ]:
function Invoke-OpenAI {
    param(
        [Parameter(Mandatory = $true)]
        $messages
    )
    
    $data = @{
        "model"       = $openai_api_model;
        "messages"    = $messages;
        "functions"   = $functions;
        "max_tokens"  = 64;
    }
    
    $headers = @{
        "Content-Type"  = "application/json"
        "Authorization" = "Bearer " + $openai_api_key
    }
    $response = Invoke-WebRequest -Uri $openai_api_endpoint -Method Post -Headers $headers -Body ($data | ConvertTo-Json -Depth 10)

    return $response;

}

### Get-Result
This function parses the response from OpenAI API and returns the result.

As we will see later, `Invoke-OpenAI` will be called first with the user's message along with information that we have a function called `Get-TubeStatus` in our code.
If the response from OpenAI specifies to call a function (in this case `Get-TubeStatus`), then `Get-TubeStatus` will be called.
The response from `Get-TubeStatus` will be passed to OpenAI again.
OpenAI will process that request, along with user's message, and return the response.

#### Example
User's message: "What is the status of the Central line?"
OpenAI's first response: "call the Get-TubeStatus Central"
`Get-TubeStatus` will be called with parameter "Central" to get the status of the Central line.
That response will be passed to OpenAI again.
OpenAI will process that response, along with user's message, and return the response.
OpenAI's final response: "The Central line is running with minor delays."


In [ ]:
function Get-Result {
    param(
        [Parameter(Mandatory = $true)]
        [object]$response,
        [Parameter(Mandatory = $true)]
        $messages
    )

    $response_json_first_choice = ($response.Content | ConvertFrom-Json | Select-Object -ExpandProperty "choices")[0];

    $response_finish_reason = $response_json_first_choice.finish_reason;

    # if response_finish_reason is "function_call" then call the function which is in $response_json_first_choice.message.function_call.name
    # and pass the arguments which is in $response_json_first_choice.message.function_call.arguments

    if ($response_finish_reason -eq "function_call") {
        $function_name = $response_json_first_choice.message.function_call.name
        $function_arguments = $response_json_first_choice.message.function_call.arguments | ConvertFrom-Json

        # Based on the function name, call the function and pass the arguments

        switch ($function_name) {
            "getTubeStatus" {
                $line = $function_arguments | Select-Object -ExpandProperty "line";
                $tube_status = Get-TubeStatus -line $line;
                
                # Compose a message of type "assistant" with function name and arguments
                $assistant_message = @{
                    "role"          = "assistant";
                    "content"       = "";
                    "function_call" = @{
                        "name"      = $function_name;
                        "arguments" = $function_arguments | ConvertTo-Json;
                    }
                }

                # Push the assistant message to messages array
                $messages += $assistant_message;

                # Compose a message of type "function" with the response from TfL API
                $function_message = @{
                    "role"    = "function";
                    "name"    = $function_name;
                    "content" = $tube_status.Content | ConvertFrom-Json | ForEach-Object { $_.name + ": " + ($_.lineStatuses[0] | ConvertTo-Json) } | Out-String;
                }

                # Push the function message to messages array
                $messages += $function_message;
            }
        }

        # Call Invoke-OpenAI function again with updated messages array
        $second_response = Invoke-OpenAI -messages $messages;
        return Get-Result -response $second_response -messages $messages;

    }

    # if response_finish_reason is "stop" then get the response from response["choices"][0]["message"]["content"]
    # and return the response

    if ($response_finish_reason -eq "stop") {
        $result = $response.Content | ConvertFrom-Json | Select-Object -ExpandProperty choices | Select-Object -ExpandProperty message | Select-Object -ExpandProperty content;
        return $result;
    }
}

### Start-Chat
This function starts the chat with the user and calls `Invoke-OpenAI` and `Get-Result` functions.

1. Get user's message
2. Call `Invoke-OpenAI` function with user's message and information that we have a function called `Get-TubeStatus` in our code
3. Call `Get-Result` function with the response from `Invoke-OpenAI` function
4. If the response from `Get-Result` function is not empty, then print the response

In [ ]:
function Start-Chat {

    param(
        [Parameter(Mandatory = $true)]
        [object]$user_input
    )

    $messages = @()
    
    # Add a system message to messages array
    $system_message = @{}
    $system_message["role"] = "system";
    $system_message["content"] = "You are a TfL Assistant. You need to help customers with their queries about London Underground.";

    Write-Host "Your question to the TfL Assistant: $user_input" -ForegroundColor Yellow
    

    # Compose a message of type "user" with user message
    $user_message = @{}
    $user_message["role"] = "user";
    $user_message["content"] = $user_input;

    # Push the user message to messages array
    $messages += $user_message;

    # Call Invoke-OpenAI function with messages array
    $response = Invoke-OpenAI -messages $messages;

    # Call Get-Result function with response and messages array
    $final_response = Get-Result -response $response -messages $messages;

    Write-Host $final_response -ForegroundColor Green
}

## A global variable to store functions array
We will be using a global variable to store the functions array.

We have only one function in the array, `getTubeStatus`.
The definition of the funtions array needs to be of a particular format.
Each function in the array needs to have a `name`, `description` and `parameters` properties.

The idea being, when this array is passed to OpenAI along with user's message, OpenAI can then read the details in this array and respond to us saying which function we need to call and what arguments we need to pass to that function.

### Example
User's message: "What is the status of the Central line?"
OpenAI's response will be something like below
```json
...
"choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": null,
                "function_call": {
                    "name": "getTubeStatus",
                    "arguments": "{\n  \"line\": \"central\"\n}"
                }
            },
            "finish_reason": "function_call"
        }
    ]
...
```

In [ ]:
$functions = @()

# Add a function to functions array
$getTubeStatusFunction = @{
    "name"        = "getTubeStatus";
    "description" = "Get the status of a London Underground line";
    "parameters"  = @{
        "type"       = "object";
        "properties" = @{
            "line" = @{
                "type"        = "string";
                "description" = "The name of the London Underground line";
                "enum"        = @("bakerloo", "central", "circle", "district", "dlr", "elizabeth", "hammersmith-city", "jubilee", "london-overground", "metropolitan", "northern", "piccadilly", "tram", "victoria", "waterloo-city");
            }
        }
        "required"   = @("line");
    }
}

$functions += $getTubeStatusFunction;

## OpenAI and TfL API using Function calling
Call `Start-Chat` function to start the chat with the user.

In [ ]:
$question = "Are there any problems on the piccadilly line?"
Start-Chat -user_input $question